# Revised case normalization for Winterthur 2019

Notes:

- 90 cases can be matched with the DB (if ignoring patientID), otherwise 57.

In [1]:
import sys
sys.path.insert(0, '/home/jovyan/work')

from src.revised_case_normalization.notebook_functions.global_configs import *
from src.revised_case_normalization.notebook_functions.normalize import normalize
from src.revised_case_normalization.notebook_functions.revise import revise
from src.revised_case_normalization.notebook_functions.group import group
from src.revised_case_normalization.notebook_functions.update_db import update_db

In [2]:
file_info = FileInfo(
         os.path.join(ROOT_DIR, 'raw_data/Winterthur.xlsx'),
         'KSW',
         '2019',
         'Änderungen Winterthur 2019')

print(file_info)



FileInfo(path='/home/jovyan/work/src/revised_case_normalization/raw_data/Winterthur.xlsx', hospital_name_db='KSW', year='2019', sheet='Änderungen Winterthur 2019')


In [3]:

columns_to_rename = dict(COLUMNS_TO_RENAME)
columns_to_rename.pop("admno")
columns_to_rename['fid'] = CASE_ID_COL

revised_cases_df = normalize(file_info, columns_mapper = columns_to_rename)
revised_cases_df

2022-11-15 14:15:23.132 | INFO     | src.revised_case_normalization.notebook_functions.normalize:normalize:35 - Read 90 cases for KSW 2019


ValueError: The following columns to rename did not exist: ['FID']. Available columns are ['tranche', 'datum', 'fid', 'patid', 'kkik', 'geschlecht', 'alter (jahre)', 'fab', 'pflegetage alt', 'pflegetage neu', 'pccl alt', 'pccl neu', 'hd alt', 'hd neu', 'hinzugefügte icd', 'gelöschte icd', 'hinzugefügte ops', 'gelöschte ops', 'drg alt', 'drg neu', 'cw alt', 'cw neu', 'cw-änderung vorschlag', 'cw änderung akzeptiert', 'kommentar dtod', 'kommentar ksw', 'angaben in welchem bericht (drg relevante diagnosen)', 'arztanfragen', 'unnamed: 28']

# Match revised DtoD data with the BfS data from the database

In [ ]:
cols_to_join = list(VALIDATION_COLS)
cols_to_join.remove(CASE_ID_COL)
cols_to_join.append(NORM_CASE_ID_COL)
# Ignore the patient ID in this dataset, because it is encrypted
cols_to_join.remove(PATIENT_ID_COL)

revised_cases, unmatched = revise(file_info, revised_cases_df, validation_cols=cols_to_join)
revised_cases.head()

In [ ]:
if unmatched.shape[0] > 0:
    display(unmatched)

In [ ]:
revisions_update, diagnoses_update, procedures_update = group(revised_cases)
revisions_update[REVISION_DATE_COL] = '2022-12-31'

In [ ]:
revisions_update

In [ ]:
update_db(revisions_update, diagnoses_update, procedures_update)